<a href="https://colab.research.google.com/github/Joshmen06/Bagging_Boosting-Project/blob/main/1_Bagging_Boosting_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import zscore

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='white', color_codes=True)

In [140]:
data = pd.read_csv("/content/talking_data.csv")

In [141]:
data.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [142]:
print("Dimension of dataset:", data.shape)
print("Number of rows in the dataset:", data.shape[0])
print("Number of columns in the dataset:", data.shape[1])

Dimension of dataset: (100000, 8)
Number of rows in the dataset: 100000
Number of columns in the dataset: 8


In [143]:
print("Column Names:",data.columns.values)

Column Names: ['ip' 'app' 'device' 'os' 'channel' 'click_time' 'attributed_time'
 'is_attributed']


In [144]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   ip               100000 non-null  int64 
 1   app              100000 non-null  int64 
 2   device           100000 non-null  int64 
 3   os               100000 non-null  int64 
 4   channel          100000 non-null  int64 
 5   click_time       100000 non-null  object
 6   attributed_time  227 non-null     object
 7   is_attributed    100000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 6.1+ MB


In [145]:
data.describe()

,ip,app,device,os,channel,is_attributed
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,91255.879670,12.04788,21.771250,22.818280,268.832460,0.002270
std,69835.553661,14.94150,259.667767,55.943136,129.724248,0.047591
min,9.000000,1.00000,0.000000,0.000000,3.000000,0.000000
25%,40552.000000,3.00000,1.000000,13.000000,145.000000,0.000000
50%,79827.000000,12.00000,1.000000,18.000000,258.000000,0.000000
75%,118252.000000,15.00000,1.000000,19.000000,379.000000,0.000000
max,364757.000000,551.00000,3867.000000,866.000000,498.000000,1.000000


In [146]:
data.isnull().sum()

ip                     0
app                    0
device                 0
os                     0
channel                0
click_time             0
attributed_time    99773
is_attributed          0
dtype: int64

In [147]:
data.value_counts()

ip      app  device  os  channel  click_time           attributed_time      is_attributed
2948    45   1       2   419      2017-11-09 14:30:44  2017-11-09 14:30:54  1                1
257270  19   0       29  213      2017-11-08 14:56:48  2017-11-08 14:58:38  1                1
220571  71   1       25  3        2017-11-08 04:35:21  2017-11-08 04:37:46  1                1
221917  10   1       7   113      2017-11-08 09:09:20  2017-11-08 09:09:41  1                1
224120  19   0       29  213      2017-11-08 02:22:13  2017-11-08 02:22:38  1                1
                                                                                            ..
115265  35   1       17  21       2017-11-08 05:48:13  2017-11-08 06:04:27  1                1
115655  19   0       29  213      2017-11-09 00:16:38  2017-11-09 00:18:24  1                1
116718  19   0       24  213      2017-11-07 10:05:58  2017-11-07 11:22:32  1                1
116785  19   0       0   213      2017-11-08 23:38:58  

In [148]:
data=data.drop('attributed_time', axis=1)

In [149]:
data.isnull().sum()

ip               0
app              0
device           0
os               0
channel          0
click_time       0
is_attributed    0
dtype: int64

In [150]:
cols=data.columns

In [151]:
for col in cols:
  unique=len(data[col].unique())
  print("Column",col, "has ",unique," unique values")

Column ip has  34857  unique values
Column app has  161  unique values
Column device has  100  unique values
Column os has  130  unique values
Column channel has  161  unique values
Column click_time has  80350  unique values
Column is_attributed has  2  unique values


In [152]:
data.app

0        12
1        25
2        12
3        13
4        12
         ..
99995    11
99996     9
99997     3
99998    15
99999    14
Name: app, Length: 100000, dtype: int64

In [153]:
app_id=data["app"].value_counts()
print(app_id)

3      18279
12     13198
2      11737
9       8992
15      8595
       ...  
145        1
97         1
96         1
267        1
261        1
Name: app, Length: 161, dtype: int64


In [154]:
total_click=app_id.sum()
print("Total clicks on app is ",app_id.sum())

Total clicks on app is  100000


In [155]:
limit=total_click*0.02

In [156]:
uapid=data.app.unique()

In [157]:
print(limit)

2000.0


In [158]:
x=app_id.values
greater=[]
less=[]
for i in range(len(app_id)):
  if x[i]>limit:
    #print(x[i])
    y=uapid[i]
    greater.append(y)
  else:
    y=uapid[i]
    less.append(y)

print("frequebtly used app id :",greater)
print()
print("App id Less than 2% of click",less)

frequebtly used app id : [12, 25, 13, 3, 1, 9, 2, 6, 20, 14]

App id Less than 2% of click [64, 7, 15, 18, 11, 8, 23, 21, 56, 28, 36, 22, 26, 17, 27, 10, 60, 58, 19, 32, 24, 4, 53, 125, 204, 5, 202, 70, 47, 29, 55, 103, 35, 83, 208, 146, 161, 46, 62, 65, 110, 42, 150, 266, 151, 148, 72, 310, 93, 94, 71, 43, 39, 170, 37, 68, 54, 107, 398, 160, 30, 183, 315, 45, 121, 38, 82, 134, 33, 233, 100, 44, 109, 59, 74, 61, 104, 66, 67, 273, 192, 50, 118, 372, 48, 232, 165, 91, 123, 88, 538, 49, 16, 105, 536, 216, 425, 117, 551, 99, 86, 101, 394, 78, 95, 181, 76, 34, 124, 548, 108, 85, 145, 79, 97, 75, 52, 215, 84, 96, 81, 139, 267, 80, 137, 119, 474, 158, 293, 122, 112, 149, 363, 271, 31, 171, 87, 168, 176, 347, 163, 190, 486, 133, 302, 268, 407, 116, 92, 115, 261]


In [159]:
data["click_time"]

0        2017-11-07 09:30:38
1        2017-11-07 13:40:27
2        2017-11-07 18:05:24
3        2017-11-07 04:58:08
4        2017-11-09 09:00:09
                ...         
99995    2017-11-09 13:25:41
99996    2017-11-07 11:25:43
99997    2017-11-08 11:38:42
99998    2017-11-08 17:55:21
99999    2017-11-07 14:32:27
Name: click_time, Length: 100000, dtype: object

In [160]:
df=data.copy()

In [161]:
# from datetime import datetime
# df.click_time=pd.to_datetime(df.click_time)

In [162]:
# df["month"]=pd.DatetimeIndex(df.click_time).month
# df["dayof_week"]=pd.DatetimeIndex(df.click_time).dayofweek
# df["day_of_year"]=pd.DatetimeIndex(df.click_time).dayofyear
# df["hour"]=pd.DatetimeIndex(df.click_time).hour

In [163]:
def timefeatures(df):
  df['datetime']=pd.to_datetime(df['click_time'])
  df['dayof_week']=df['datetime'].dt.dayofweek
  df['day_of_year']=df['datetime'].dt.dayofyear
  df['month']=df['datetime'].dt.month
  df['hour']=df['datetime'].dt.hour
  return df

In [164]:
df=timefeatures(df)
df.drop(['click_time','datetime'],axis=1,inplace=True)
df.head()

,ip,app,device,os,channel,is_attributed,dayof_week,day_of_year,month,hour
0,87540,12,1,13,497,0,1,311,11,9
1,105560,25,1,17,259,0,1,311,11,13
2,101424,12,1,19,212,0,1,311,11,18
3,94584,13,1,13,477,0,1,311,11,4
4,68413,12,1,1,178,0,3,313,11,9


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype
---  ------         --------------   -----
 0   ip             100000 non-null  int64
 1   app            100000 non-null  int64
 2   device         100000 non-null  int64
 3   os             100000 non-null  int64
 4   channel        100000 non-null  int64
 5   is_attributed  100000 non-null  int64
 6   dayof_week     100000 non-null  int64
 7   day_of_year    100000 non-null  int64
 8   month          100000 non-null  int64
 9   hour           100000 non-null  int64
dtypes: int64(10)
memory usage: 7.6 MB


In [166]:
x=df.drop(['is_attributed'],axis=1)
y=df['is_attributed']


In [167]:
y

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: is_attributed, Length: 100000, dtype: int64

In [168]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8)
print(x_train.shape)
print(x_test.shape)

(80000, 9)
(20000, 9)


In [169]:
from sklearn import metrics
from sklearn import preprocessing

In [171]:
from sklearn.ensemble import GradientBoostingClassifier
gbcl = GradientBoostingClassifier(n_estimators = 10, learning_rate = 0.05)
gbcl = gbcl.fit(x_train, y_train)
test_pred = gbcl.predict(x_test)
gbcl_score=gbcl.score(x_test , y_test)
print("The Gradient Boost classifier model accuracy is {}".format(gbcl_score))
print("The confusion matrix of Gradient Boost classifier model is")
print(metrics.confusion_matrix(y_test,test_pred))

The Gradient Boost classifier model accuracy is 0.9976
The confusion matrix of Gradient Boost classifier model is
[[19950     2]
 [   46     2]]


In [170]:
from sklearn.ensemble import BaggingClassifier
bgcl=BaggingClassifier(n_estimators=10,bootstrap=True,max_samples=.7)
bgcl=bgcl.fit(x_train, y_train)
test_pred=bgcl.predict(x_test)
bgcl_score=bgcl.score(x_test, y_test)
print('the bagging classoifier model accuracy is {}'.format(bgcl_score))
print('the confusion matrix of bagging classifier model is')
print(metrics.confusion_matrix(y_test,test_pred))

the bagging classoifier model accuracy is 0.99785
the confusion matrix of bagging classifier model is
[[19946     6]
 [   37    11]]


In [172]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [173]:
Y_pred=bgcl.predict_proba(x_test)
Y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [175]:
auc=metrics.roc_auc_score(y_test,Y_pred[:,1])
print(auc)

0.7469055942595563


In [176]:
import xgboost as xgb
from xgboost import XGBClassifier
xgbl=XGBClassifier()
xgbl.fit(x_train,y_train)
test_pred=xgbl.predict(x_test)
xgbl_score=xgbl.score(x_test,y_test)
print(xgbl_score)

0.9978


In [177]:
print(metrics.confusion_matrix(y_test,test_pred))

[[19952     0]
 [   44     4]]
